In [1]:
import os
import logging
import math
from filelock import FileLock
import random

# __import_lightning_begin__
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import FashionMNIST
from torchvision import transforms
# __import_lightning_end__

# __import_tune_begin__
from pytorch_lightning.loggers import TensorBoardLogger
from ray import tune
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
# __import_tune_end__

from MNISTDataModule import MNISTDataModule
from augmentation import TRANSFORM_NAMES
from LightningFashionMNIST import LightningFashionMNIST


In [2]:
log = logging.getLogger('App')
logging.basicConfig(level=logging.INFO)

In [3]:
def train_mnist_tune_checkpoint(config,
                                checkpoint_dir=None,
                                # no really used, we stop after every epoch and let tune decide what to do
                                num_epochs=999,
                                num_gpus=0):
    # data_dir = os.path.expanduser("~/data")
    data_dir = config["data_dir"]

    trainer = pl.Trainer(
        max_epochs=num_epochs,
        # If fractional GPUs passed in, convert to int.
        gpus=math.ceil(num_gpus),
        logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version="."),
        progress_bar_refresh_rate=config["progress_bar_refresh_rate"],
        num_sanity_val_steps=0,
        callbacks=[
            TuneReportCheckpointCallback(
                metrics={
                    "loss": "ptl/val_loss",
                    "mean_accuracy": "ptl/val_acc"
                },
                filename="checkpoint",
                on="validation_end"
            )
        ]
    )

    if checkpoint_dir:
        model = LightningFashionMNIST.load_from_checkpoint(os.path.join(checkpoint_dir, "checkpoint"), conf=config)
        log.info('Lightning loaded from checkpoint')
    else:
        model = LightningFashionMNIST(conf=config)
        log.info('Lightning initialized')

    data = MNISTDataModule(conf=config)

    trainer.fit(model, data)

In [4]:
def tune_mnist_pbt(num_samples=15, training_iteration=15, gpus_per_trial=0):
    def explore(config):
        log.info("======================================= EXPLORE =========================================")
        # calculate new magnitudes for augmentations
        augmentations = []
        for tfn_name in TRANSFORM_NAMES:
            augmentations.append((tfn_name, random.random()))

        config["augmentations"] = augmentations
        log.info(config)
        return config

    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        # Models will be considered for perturbation at this interval of time_attr="time_total_s"
        perturbation_interval=1,
        custom_explore_fn=explore,
        log_config=True
    )

    reporter_jupyter = JupyterNotebookReporter(
        overwrite=True,
        parameter_columns=["augmentations"],
        metric_columns=["loss", "mean_accuracy", "training_iteration"]
    )

    """
    # initial config
            config={
            "progress_bar_refresh_rate": 0,
            "layer_1_size": tune.choice([32, 64, 128, 256, 512, 1024]),
            "layer_2_size": tune.choice([32, 64, 128, 256, 512, 1024]),
            "lr": tune.choice([1e-2, 1e-3, 1e-4, 1e-5, 1e-6]),
            "batch_size": tune.choice([32, 64, 128, 256, 512, 1024, 2048]),
        },
    """

    # set up the augmentations
    # tuple of augmentation name and its magnitude
    augmentations = []
    for tfn_name in TRANSFORM_NAMES:
        augmentations.append((tfn_name, random.random()))

    config={
        # https://docs.ray.io/en/master/tune/api_docs/search_space.html?highlight=tune.choice#
        "progress_bar_refresh_rate": 0,
        "layer_1_size": 512,
        "layer_2_size": 512,
        "lr": 0.00005 ,
        "batch_size": 1024,
        "data_dir": "./data",
        "data_mean": 0.28604063391685486,
        "data_std": 0.35302430391311646,
        "augmentations": augmentations,
    }

    analysis = tune.run(
        tune.with_parameters(
            train_mnist_tune_checkpoint,
            num_gpus=gpus_per_trial),
        resources_per_trial={
            "cpu": 1 / 16,
            "gpu": gpus_per_trial
        },
        metric="loss",
        mode="min",
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter_jupyter,
        verbose=1,
        name="FashionMNIST-pbt",
        stop={  # Stop a single trial if one of the conditions are met
            "mean_accuracy": 0.95,
            "training_iteration": training_iteration},
        local_dir="./data",
    )

    print("Best hyperparameters found were: ", analysis.best_config)
    return analysis

In [7]:
analysis = tune_mnist_pbt(num_samples=30, training_iteration=15, gpus_per_trial=1/8)
analysis.best_config
analysis.results

(pid=393745) /home/akaver/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/deprecated_api.py:25: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
(pid=393745)   rank_zero_deprecation(
2021-10-12 01:26:59,106	INFO tune.py:550 -- Total run time: 5502.55 seconds (5502.40 seconds for the tuning loop).


Best hyperparameters found were:  {'progress_bar_refresh_rate': 0, 'layer_1_size': 512, 'layer_2_size': 512, 'lr': 5e-05, 'batch_size': 1024, 'data_dir': './data', 'data_mean': 0.28604063391685486, 'data_std': 0.35302430391311646, 'augmentations': [('blur', 0.2172307474348526), ('rotate_left', 0.19635165171173796), ('rotate_right', 0.21067099883267293)]}


{'898a5_00000': {'loss': 0.3831958770751953,
  'mean_accuracy': 0.8681847453117371,
  'time_this_iter_s': 90.98991799354553,
  'should_checkpoint': True,
  'done': True,
  'timesteps_total': None,
  'episodes_total': None,
  'training_iteration': 15,
  'experiment_id': 'a3e7298302b543daa77de65b3fac9e40',
  'date': '2021-10-12_01-21-37',
  'timestamp': 1633990897,
  'time_total_s': 1326.476188659668,
  'pid': 386525,
  'hostname': 'ml-linux',
  'node_ip': '192.168.1.23',
  'config': {'progress_bar_refresh_rate': 0,
   'layer_1_size': 512,
   'layer_2_size': 512,
   'lr': 5e-05,
   'batch_size': 1024,
   'data_dir': './data',
   'data_mean': 0.28604063391685486,
   'data_std': 0.35302430391311646,
   'augmentations': [('blur', 0.26791761175640405),
    ('rotate_left', 0.8723674737797285),
    ('rotate_right', 0.07890809261122134)]},
  'time_since_restore': 90.98991799354553,
  'timesteps_since_restore': 0,
  'iterations_since_restore': 1,
  'trial_id': '898a5_00000',
  'experiment_tag': 

In [5]:
from time import sleep, perf_counter as pc

results = []
for models in range(8):
    experiments_to_run_in_parallel = (models + 1) * 2
    print(f"Starting {experiments_to_run_in_parallel} experiments in parallel")
    start_time = pc()

    analysis = tune_mnist_pbt(num_samples=16, num_epochs=5, gpus_per_trial=1 / experiments_to_run_in_parallel)

    elapsed_time = pc() - start_time
    results.append((models, elapsed_time))

results

Trial name,# failures,error file
train_mnist_tune_checkpoint_575ec_00000,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00000_0_batch_size=256,layer_1_size=1024,layer_2_size=128,lr=0.0001_2021-09-28_01-29-13/error.txt"
train_mnist_tune_checkpoint_575ec_00001,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00001_1_batch_size=256,layer_1_size=512,layer_2_size=512,lr=1e-05_2021-09-28_01-29-13/error.txt"
train_mnist_tune_checkpoint_575ec_00002,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00002_2_batch_size=32,layer_1_size=512,layer_2_size=64,lr=0.01_2021-09-28_01-29-13/error.txt"
train_mnist_tune_checkpoint_575ec_00003,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00003_3_batch_size=64,layer_1_size=512,layer_2_size=1024,lr=1e-05_2021-09-28_01-29-13/error.txt"
train_mnist_tune_checkpoint_575ec_00004,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00004_4_batch_size=1024,layer_1_size=1024,layer_2_size=64,lr=0.01_2021-09-28_01-29-14/error.txt"
train_mnist_tune_checkpoint_575ec_00005,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00005_5_batch_size=32,layer_1_size=1024,layer_2_size=64,lr=0.0001_2021-09-28_01-29-14/error.txt"
train_mnist_tune_checkpoint_575ec_00006,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00006_6_batch_size=512,layer_1_size=64,layer_2_size=1024,lr=0.01_2021-09-28_01-29-14/error.txt"
train_mnist_tune_checkpoint_575ec_00007,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00007_7_batch_size=256,layer_1_size=1024,layer_2_size=64,lr=0.0001_2021-09-28_01-29-14/error.txt"
train_mnist_tune_checkpoint_575ec_00008,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00008_8_batch_size=512,layer_1_size=512,layer_2_size=256,lr=1e-06_2021-09-28_01-29-14/error.txt"
train_mnist_tune_checkpoint_575ec_00009,1,"/home/akaver/!Dev/pbt-demo-mnist/data/FashionMNIST/train_mnist_tune_checkpoint_575ec_00009_9_batch_size=2048,layer_1_size=32,layer_2_size=128,lr=1e-06_2021-09-28_01-29-14/error.txt"


TuneError: ('Trials did not complete', [train_mnist_tune_checkpoint_575ec_00000, train_mnist_tune_checkpoint_575ec_00001, train_mnist_tune_checkpoint_575ec_00002, train_mnist_tune_checkpoint_575ec_00003, train_mnist_tune_checkpoint_575ec_00004, train_mnist_tune_checkpoint_575ec_00005, train_mnist_tune_checkpoint_575ec_00006, train_mnist_tune_checkpoint_575ec_00007, train_mnist_tune_checkpoint_575ec_00008, train_mnist_tune_checkpoint_575ec_00009, train_mnist_tune_checkpoint_575ec_00010, train_mnist_tune_checkpoint_575ec_00011, train_mnist_tune_checkpoint_575ec_00012, train_mnist_tune_checkpoint_575ec_00013, train_mnist_tune_checkpoint_575ec_00014, train_mnist_tune_checkpoint_575ec_00015])

In [6]:
results

[(0, 1250.4137557320064),
 (1, 1054.7564893119998),
 (2, 978.0088943799929),
 (3, 1215.7183314590075),
 (4, 1026.1449859720015),
 (5, 1067.1325380739872),
 (6, 1058.9811587199947)]